# Notebook: Frustrationskorrektur zu $\alpha^{-1}$
Dieses Notebook rekonstruiert die im Kurzpapier angegebenen Zahlen, fuehrt Z-Tests gegen CODATA 2022 sowie PDG 2024 durch.

In [1]:
# Globale Ausgabesteuerung (harmonisiert)
import os, pathlib, hashlib, json, datetime, hashlib, mpmath as mp, sympy as sp 
from dataclasses import dataclass

mp.mp.dps = max(mp.mp.dps, 80) # Dezimalstellen für mpmath

def write_sha256(path):
    sha=hashlib.sha256(open(path,"rb").read()).hexdigest()
    open(path+".sha256","w").write(sha+"  "+os.path.basename(path)+"\n")
    return sha
    
def _best_outdir(preferred):
    p = pathlib.Path(preferred)
    p.mkdir(parents=True, exist_ok=True)
    return p

OUTDIR = _best_outdir("anc")
os.makedirs(OUTDIR, exist_ok=True)
print(f"[Alpha] OUTDIR = {OUTDIR.resolve()}")


[Alpha] OUTDIR = /Users/mcadmin/Documents/Entwicklung/alpha0/notebooks/anc


In [2]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, ClassVar, Any, Iterable  # ← Iterable ergänzt

try:
    mp
except NameError:
    raise RuntimeError("Obere Zellen müssen erst ausgeführt sein")

mp.mp.dps = max(mp.mp.dps, 80)

def _to_mpf(x: Any) -> "mp.mpf":
    """Robust nach mp.mpf konvertieren; vermeidet stille float-Rundung via str-Fallback."""
    if isinstance(x, (mp.mpf, int)):
        return mp.mpf(x)
    if sp is not None and isinstance(x, (sp.Float, sp.Integer, sp.Rational)):
        return mp.mpf(str(x))
    return mp.mpf(str(x))

def _nstr(x: "mp.mpf", prec: int = 30) -> str:
    return mp.nstr(x, prec)

@dataclass(frozen=True, slots=True)
class PhysQuantity:
    name: str
    value: mp.mpf
    symbol: Optional[str] = None
    unit: Optional[str] = None          # None ⇒ "1" in der Ausgabe
    uncertainty: Optional[mp.mpf] = None

    def __post_init__(self):
        object.__setattr__(self, "value", _to_mpf(self.value))
        if self.uncertainty is not None:
            object.__setattr__(self, "uncertainty", _to_mpf(self.uncertainty))

    # Properties
    @property
    def v(self) -> "mp.mpf":
        return self.value

    @property
    def sigma(self) -> Optional["mp.mpf"]:
        return self.uncertainty

    # z = (self - ref) / σ_ref ; None falls σ_ref fehlt oder 0 ist
    def z_score(self, ref_q: "PhysQuantity") -> Optional["PhysQuantity"]:
        if (ref_q.sigma is None) or (ref_q.sigma == 0):
            return None
        return PhysQuantity(
            name   = f"z-Score {self.name} zu {ref_q.name}",
            symbol = "z",
            value  = (self.v - ref_q.v) / ref_q.sigma,
        )

    # Ausgabe
    def _display_symbol(self) -> str:
        return self.symbol if self.symbol is not None else "v"

    def _display_unit(self) -> str:
        return "1" if self.unit is None else self.unit

    def __str__(self) -> str:
        s = self.sigma
        sym = self._display_symbol()
        unit = self._display_unit()
        unc = "" if s is None else f" ± {_nstr(s)}"
        unit_str = f" {unit}" if unit != "1" else ""
        return f"{sym} = {_nstr(self.v)}{unc}{unit_str}".strip()

    # --- Serialisierung ---
    @classmethod
    def headers(cls, abs_unc: bool = False, labeled: bool = False) -> tuple[str, ...]:
        keys = ["name", "symbol", "value", "uncertainty_rel", "unit"]
        if abs_unc:
            keys.insert(3, "uncertainty")  # vor 'uncertainty_rel'
        if not labeled:
            return tuple(keys)
        labels = {
            "name": "Grösse", "symbol": "Symbol", "value": "Wert",
            "uncertainty": "σ", "uncertainty_rel": "σ/Wert [%]", "unit": "Einheit",
        }
        return tuple(labels[k] for k in keys)

    def to_dict(self, prec: int = 30, latex: bool = True,
                abs_unc: bool = False, prec_sigma: int = 5) -> dict[str, str]:
        sym  = self._display_symbol()
        unit = self._display_unit()
        if latex:
            sym  = f"${sym}$"
            unit = f"${unit}$"

        if self.sigma is None:
            sig_abs_str = ""
            sig_rel_str = ""
        else:
            sig_abs_str = _nstr(self.sigma, prec) if abs_unc else ""
            sig_rel_str = _nstr(100 * self.sigma / self.v, prec_sigma)

        return {
            "name": self.name,
            "symbol": sym,
            "value": _nstr(self.v, prec),
            "uncertainty": sig_abs_str,
            "uncertainty_rel": sig_rel_str,
            "unit": unit,
        }

    def row(self, prec: int = 30, abs_unc: bool = True, latex: bool = True) -> list[str]:
        d = self.to_dict(prec=prec, latex=latex, abs_unc=abs_unc, prec_sigma=5)
        return [d[k] for k in self.headers(abs_unc=abs_unc)]

    # Vergleichszeile: self (theo) vs. ref (Messwert)
    def row_benchmark(self, ref: "PhysQuantity",
                      prec: int = 12, prec_sigma: int = 5) -> list[str]:
        z = self.z_score(ref)
        return [
            ref.name,
            _nstr(ref.v, prec),
            (_nstr(100 * ref.sigma / ref.v, prec_sigma) if ref.sigma is not None else ""),
            _nstr(self.v, prec),
            (_nstr(100 * (self.v - ref.v) / ref.v, prec_sigma) if ref.v != 0 else ""),
            (_nstr(z.v, prec_sigma) if z is not None else ""),
        ]

# Utilities
def require(name: str, cond: bool):
    if not cond:
        raise RuntimeError(f"Voraussetzung fehlt/verletzt: {name}")

def md_table(headers: Iterable[str], rows: Iterable[Iterable[Any]], path: Optional[str] = None) -> str:
    """Nur Spalten aus `headers` ausgeben; Zeilen auf/abfüllen."""
    def isnum(s: str) -> bool:
        s2 = str(s).strip()
        if s2 == "":
            return False
        try:
            _ = mp.mpf(s2)
            return True
        except Exception:
            return False

    headers = list(headers)
    cols = len(headers)

    norm_rows: list[list[str]] = []
    for r in rows:
        rr = [("" if c is None else str(c)) for c in r]
        if len(rr) < cols:
            rr += [""] * (cols - len(rr))
        elif len(rr) > cols:
            rr = rr[:cols]
        norm_rows.append(rr)

    align = []
    for j in range(cols):
        col = [r[j] for r in norm_rows] if norm_rows else []
        nums = [x for x in col if x != ""]
        align.append("right" if nums and all(isnum(x) for x in nums) else "left")

    widths = [len(h) for h in headers]
    for r in norm_rows:
        for j in range(cols):
            if len(r[j]) > widths[j]:
                widths[j] = len(r[j])

    def align_token(j: int) -> str:
        w = max(3, widths[j] + 2)
        if align[j] == "left":
            return ":" + "-" * (w - 1)
        elif align[j] == "right":
            return "-" * (w - 1) + ":"
        inner = max(1, w - 2)
        return ":" + "-" * inner + ":"

    def pad_body(s: str, j: int) -> str:
        w = widths[j]
        if align[j] == "right":
            return s.rjust(w)
        if align[j] == "center":
            pad = w - len(s); L = pad // 2
            return " " * L + s + " " * (pad - L)
        return s.ljust(w)

    header = "| " + " | ".join(headers[j].ljust(widths[j]) for j in range(cols)) + " |"
    alrow  = "|"  +  "|".join(align_token(j) for j in range(cols)) + "|"
    body   = ["| " + " | ".join(pad_body(r[j], j) for j in range(cols)) + " |" for r in norm_rows]

    md = "\n".join([header, alrow] + body) + "\n"
    if path:
        with open(path, "w", encoding="utf-8") as f:
            f.write(md)
    return md

In [3]:
# ========================================
# Konstanten & Basisgrössen (PhysQuantity, mpmath)
# ========================================
# Sicherstellen, dass mpmath initialisiert ist
try:
    PhysQuantity
except NameError:
    raise RuntimeError("Obere Zellen müssen erst ausgeführt sein")

# Basiszahlen (Code: N1/N8/N12; Symbol: N_i)
N1  = PhysQuantity(name="1",            symbol="N_1",   value=1)
N8  = PhysQuantity(name="8",            symbol="N_8",   value=8)
N12 = PhysQuantity(name="12",           symbol="N_12",  value=12)

# Primzahlen (Code: P2/P3/…; Symbol: P_j)
P2  = PhysQuantity(name="2 (Primzahl)",  symbol="P_2",    value=2)
P3  = PhysQuantity(name="3 (Primzahl)",  symbol="P_3",    value=3)
P5  = PhysQuantity(name="5 (Primzahl)",  symbol="P_5",    value=5)
P7  = PhysQuantity(name="7 (Primzahl)",  symbol="P_7",    value=7)
P11 = PhysQuantity(name="11 (Primzahl)", symbol="P_{11}", value=11)

# --- Geometrische Packungsdichten ---
ETA_FCC = PhysQuantity(
    name   = "FCC-Referenzdichte (π / (3√2))",
    symbol = r"\eta_{\rm FCC}",
    value  = mp.pi / (P3.v * mp.sqrt(P2.v)),
)

ETA1 = PhysQuantity(
    name   = "Teträdrische Teilstruktur (erste Teilung; 1/2 − 1/(2·3·5·11))",
    symbol = r"\eta_1",
    value  = N1.v / P2.v - N1.v / (P2.v * P3.v * P5.v * P11.v),
)

ETA2 = PhysQuantity(
    name   = "BCC-Packungsdichte (π√3 / 8)",
    symbol = r"\eta_2",
    value  = mp.pi * mp.sqrt(mp.mpf("3")) / mp.mpf("8"),
)

# --- Teilungsbasen (bewusst klein n_i) ---
n1_BASE = PhysQuantity(name="Teilungsanteil n1 (erste Teilung)",  symbol=r"n_1", value="1")
n2_BASE = PhysQuantity(name="Teilungsanteil n2 (zweite Teilung)", symbol=r"n_2", value="1")

# --- Feinstrukturkonstante (α, nicht α^{-1}) ---
ALPHA_PDG_2024 = PhysQuantity(
    name        = "Feinstrukturkonstante, PDG 2024 World Average",
    symbol      = r"\alpha_0",
    value       = "0.007297352564278",
    unit        = "1",
    uncertainty = "0.000000000000426",
)

ALPHA_CODATA_2022 = PhysQuantity(
    name        = "Feinstrukturkonstante, CODATA 2022 (global fit)",
    symbol      = r"\alpha_0",
    value       = "0.0072973525643",
    unit        = "1",
    uncertainty = "0.0000000000011",
)

# --- Kosmologische Dichten ---
RHO_PLANCK = PhysQuantity(
    name   = "Planck-Dichte (theoretisch)",
    symbol = r"\rho_P",
    value  = "5.15484850324e96",
    unit   = "kg m^{-3}",
)

RHO_PLANCK2018 = PhysQuantity(
    name        = "Planck 2018 (CMB, ΛCDM)",
    symbol      = r"\rho_{\rm ref}",
    value       = "8.532855164e-27",
    unit        = "kg m^{-3}",
    uncertainty = "1.266002250e-28",
)

RHO_WMAP9 = PhysQuantity(
    name        = "WMAP9 + BAO + SN",
    symbol      = r"\rho_{\rm ref}",
    value       = "9.025924346e-27",
    unit        = "kg m^{-3}",
    uncertainty = "2.083306254e-28",
)

RHO_SH0ES = PhysQuantity(
    name        = "SH0ES 2021 (lokal)",
    symbol      = r"\rho_{\rm ref}",
    value       = "1.002065500e-26",
    unit        = "kg m^{-3}",
    uncertainty = "2.853636691e-28",
)

# ========================================
# Konstanten-Übersicht (Geometrie & Dichten)
# (Header & Zeilen beide mit abs. + relativer σ)
# ========================================

consts = [
    ALPHA_PDG_2024, ALPHA_CODATA_2022,
    ETA_FCC, ETA1, ETA2,
    RHO_PLANCK, RHO_PLANCK2018, RHO_WMAP9, RHO_SH0ES,
]

rows = [pq.row(prec=12, abs_unc=True) for pq in consts]
print(md_table(PhysQuantity.headers(abs_unc=True, labeled=True), rows,
               path="anc/basic_values.md"))

| Grösse                                                        | Symbol           | Wert              | σ               | σ/Wert [%] | Einheit     |
|:--------------------------------------------------------------|:-----------------|------------------:|----------------:|-----------:|:------------|
| Feinstrukturkonstante, PDG 2024 World Average                 | $\alpha_0$       |  0.00729735256428 |        4.26e-13 |  5.8377e-9 | $1$         |
| Feinstrukturkonstante, CODATA 2022 (global fit)               | $\alpha_0$       |   0.0072973525643 |         1.1e-12 |  1.5074e-8 | $1$         |
| FCC-Referenzdichte (π / (3√2))                                | $\eta_{\rm FCC}$ |    0.740480489693 |                 |            | $1$         |
| Teträdrische Teilstruktur (erste Teilung; 1/2 − 1/(2·3·5·11)) | $\eta_1$         |     0.49696969697 |                 |            | $1$         |
| BCC-Packungsdichte (π√3 / 8)                                  | $\eta_2$         |    0.6801747615

In [4]:
# ========================================
# Kernformel zu Dichte (ohne Zwischenvariablen)
# nutzt: N1, P2, P3, N8, N12, ETA_FCC, RHO_PLANCK
# ========================================

try: ETA_FCC
except NameError as e: 
    raise RuntimeError("Bitte obige Zellen zuerst ausführen.") from e

def eta_eff_em() -> PhysQuantity:
    # η_eff = 1/2·η_FCC^(4/3) + 1/2·η_FCC^2
    val = (N1.v / P2.v) * (ETA_FCC.v ** ((P2.v * P2.v) / P3.v)) \
        + (N1.v / P2.v) * (ETA_FCC.v ** P2.v)
    return PhysQuantity(
        name   = "Effektive Geometriedichte η_eff (1/2·η_FCC^(4/3) + 1/2·η_FCC^2)",
        symbol = r"\eta_{\rm eff}",
        value  = val,
    )

ETA_EFF_EM = eta_eff_em()  # einmalig setzen

# ρ_EM = ρ_P / (η_eff · N12 · N8**n8)
def rho_model_from_n8(n8: int) -> mp.mpf:
    return RHO_PLANCK.v / (ETA_EFF_EM.v * N12.v * (N8.v ** mp.mpf(n8)))

# n8 = ( ln(ρ_P/(ρ_EM·η_eff)) − ln N12 ) / ln N8
def n8_from_rho_model(rho: PhysQuantity) -> mp.mpf:
    return (mp.log(RHO_PLANCK.v / (rho.v * ETA_EFF_EM.v)) - mp.log(N12.v)) / mp.log(N8.v)

In [5]:
# ========================================
# RHO-run, z SIGNIERT via z_score
# ========================================

try: ETA_EFF_EM
except NameError as e: 
    raise RuntimeError("Bitte obige Zellen zuerst ausführen.") from e

n8_PLANCK2018 = PhysQuantity(
    name   = "Teilungsanteil n8 (Dichte-Run)",
    symbol = r"n_8",
    value  = n8_from_rho_model(RHO_PLANCK2018),
)

n3_BASE = PhysQuantity(
    name   = "Teilungsrate der 8-Fach Projektionen aus Dichte-Pfad",
    symbol = r"n_3",
    value  = int(mp.nint(n8_PLANCK2018.v)),
)

n0_FROM_PARTS = PhysQuantity(
    name   = "Teilungen aus n1, n2 und n3",
    symbol = r"n_0",
    value  = n1_BASE.v + n2_BASE.v + n3_BASE.v,
)

# Dichtemodell (Single Source of Truth)
RHO_MODEL = PhysQuantity(
    name   = f"Dichtemodell (n8 = {int(n3_BASE.v)})",
    symbol = r"\rho_{\rm model}",
    value  = rho_model_from_n8(int(n3_BASE.v)),
    unit   = RHO_PLANCK.unit,
)

RHO_PLANCK2018_z = RHO_MODEL.z_score(RHO_PLANCK2018)

print(md_table(
    ["name", "symbol", "value", "-", "-", "unit"],
    [
        n8_PLANCK2018.row(),          # relative σ wird immer ausgegeben (uncertainty_rel)
        n3_BASE.row(),
        RHO_MODEL.row(prec=12),
        RHO_PLANCK2018_z.row(prec=12),
    ],
    path="anc/density_core.md",
))


| name                                                       | symbol             | value                           | - | - | unit        |
|:-----------------------------------------------------------|:-------------------|--------------------------------:|:--|:--|:------------|
| Teilungsanteil n8 (Dichte-Run)                             | $n_8$              | 135.000105078010161766813218841 |   |   | $1$         |
| Teilungsrate der 8-Fach Projektionen aus Dichte-Pfad       | $n_3$              |                           135.0 |   |   | $1$         |
| Dichtemodell (n8 = 135)                                    | $\rho_{\rm model}$ |               8.53471982711e-27 |   |   | $kg m^{-3}$ |
| z-Score Dichtemodell (n8 = 135) zu Planck 2018 (CMB, ΛCDM) | $z$                |                 0.0147287503368 |   |   | $1$         |



In [6]:
# Guard bleibt kompakt
try: ETA1
except NameError as e:
    raise RuntimeError("Bitte obige Zellen zuerst ausführen.") from e

def Delta_eta_K(
    K:       PhysQuantity = P3,
    eta1:    PhysQuantity = ETA1,
    eta2:    PhysQuantity = ETA2,
    eta_fcc: PhysQuantity = ETA_FCC,
    n0:      PhysQuantity = n0_FROM_PARTS
) -> PhysQuantity:
    # Δη(K) = (η1/η_FCC) * (η2/η_FCC) * (2^K) / n0
    return PhysQuantity(
        name   = rf"Δη (K={K.name})",
        symbol = r"\Delta\eta",
        value  = (eta1.v/eta_fcc.v) * (eta2.v/eta_fcc.v) * (mp.mpf('2')**K.v) / n0.v,
    )

def alpha0_inv_theo_K(K: PhysQuantity) -> PhysQuantity:
    # α0^{-1}(K) = n0 + Δη(K)
    return PhysQuantity(
        name   = rf"α0^{{-1}} theo (K={K.name})",
        symbol = r"\alpha_0^{-1}",
        value  = n0_FROM_PARTS.v + Delta_eta_K(K).v,
    )

ALPHA_INV_THEO = alpha0_inv_theo_K(P3)
ALPHA_THEO = PhysQuantity(name="α0 (theo)", symbol=r"\alpha_0", value=1 / ALPHA_INV_THEO.v)

def test_e(K: PhysQuantity):
    return [rf"K = {K.name}", mp.nstr(alpha0_inv_theo_K(K).v, 12)]

_N2Test = PhysQuantity(name="2", value=2)
_N4Test = PhysQuantity(name="4", value=4)

print(md_table(
    ["K", "α0^{-1} (theo)"],
    [test_e(_N2Test), test_e(P3), test_e(_N4Test)],
    path="anc/alpha_inf_theo_compare.md",
))

print(md_table(
    PhysQuantity.headers(abs_unc=False, labeled=True),
    [Delta_eta_K(P3).row(), ETA1.row(), ETA2.row(), ETA_FCC.row(), ALPHA_INV_THEO.row(), ALPHA_THEO.row()],
    path="anc/alpha_theo_compact.md",
))

| K                | α0^{-1} (theo) |
|:-----------------|---------------:|
| K = 2            |   137.01799959 |
| K = 3 (Primzahl) |  137.035999179 |
| K = 4            |  137.071998359 |

| Grösse                                                        | Symbol           | Wert                               | σ/Wert [%] | Einheit |
|:--------------------------------------------------------------|:-----------------|-----------------------------------:|:-----------|:--------|
| Δη (K=3 (Primzahl))                                           | $\Delta\eta$     |  0.0359991793692795870417481933869 |            |         |
| Teträdrische Teilstruktur (erste Teilung; 1/2 − 1/(2·3·5·11)) | $\eta_1$         |    0.49696969696969696969696969697 |            |         |
| BCC-Packungsdichte (π√3 / 8)                                  | $\eta_2$         |   0.680174761587831693972779346616 |            |         |
| FCC-Referenzdichte (π / (3√2))                                | $\eta_{\rm FCC}$ |

In [7]:
# ========================================
# Benchmark: α0 (K=3) vs. Referenzen (PDG/CODATA)
# ========================================

try: ALPHA_THEO
except NameError as e:
    raise RuntimeError("Bitte obige Zellen zuerst ausführen.") from e

headers = ["Referenz", "α0 (ref)", "σ(α_ref)/α_ref [%]", "α0 (theo, K=3)", "Δα/α_ref [%]", "z"]
rows = [
    ALPHA_THEO.row_benchmark(ALPHA_PDG_2024),
    ALPHA_THEO.row_benchmark(ALPHA_CODATA_2022),
]

print(md_table(headers, rows, path="anc/alpha_K3_rel.md"))

# ========================================
# Dichte-Validierung – Vergleich Modell vs. kosmologischen Referenzen (ein Pfad)
# ========================================

headers = ["Referenz","ρ_ref","σ(ρ_ref) /ρ_ref [%]","ρ_model","Δρ /ρ_ref [%]","z"]
rows = [
    RHO_MODEL.row_benchmark(RHO_PLANCK2018),
    RHO_MODEL.row_benchmark(RHO_WMAP9),
    RHO_MODEL.row_benchmark(RHO_SH0ES),
]

print(md_table(headers, rows, path="anc/dichte_validation_rel.md"))


| Referenz                                        | α0 (ref)         | σ(α_ref)/α_ref [%] | α0 (theo, K=3)   | Δα/α_ref [%] | z         |
|:------------------------------------------------|-----------------:|-------------------:|-----------------:|-------------:|----------:|
| Feinstrukturkonstante, PDG 2024 World Average   | 0.00729735256428 |          5.8377e-9 | 0.00729735256421 |  -9.9683e-10 |  -0.17076 |
| Feinstrukturkonstante, CODATA 2022 (global fit) |  0.0072973525643 |          1.5074e-8 | 0.00729735256421 |   -1.2983e-9 | -0.086129 |

| Referenz                | ρ_ref           | σ(ρ_ref) /ρ_ref [%] | ρ_model           | Δρ /ρ_ref [%] | z        |
|:------------------------|----------------:|--------------------:|------------------:|--------------:|---------:|
| Planck 2018 (CMB, ΛCDM) | 8.532855164e-27 |              1.4837 | 8.53471982711e-27 |      0.021853 | 0.014729 |
| WMAP9 + BAO + SN        | 9.025924346e-27 |              2.3081 | 8.53471982711e-27 |       -5.4422 

### Zertifikat "11" – Ziel

Wir bestimmen die **minimale nichtlokale Zykluslänge** im isotropen A₂-Torus T_{L,L}.  
Nichtlokal = Zyklus mit nichttrivialer Umwicklung im Deckraum.  
Hypothese: $\ell_{\min}^{\mathrm{nonloc}} = L$.  
Für $L=11$ folgt daraus der Residualfaktor "11" in
$$
\Delta_\eta=\Bigl(1-\tfrac{1}{3\cdot5\cdot11}\Bigr)\frac{9\sqrt{3}}{\pi n_0}.
$$

In [8]:
# Zertifikat "11" – Sichtbarer Beweis (Notebook, eine Zelle)
# A: Schranke ≥ L (informativ), B: Witness für L=11, C: BFS-Minimalitätszertifikat,
#    dazu Tabelle (3..15), CSV inline, SHA-256-Hash und Beweis-Legende.

from collections import deque
import pandas as pd, hashlib

# Schritte im A2-Gitter (±e1, ±e2, ±(e1−e2))
STEPS = [(1,0),(-1,0),(0,1),(0,-1),(1,-1),(-1,1)]

def min_nonlocal_length_bfs(L, record_levels=False):
    """Kleinste Länge eines geschlossenen Pfades, der (0,0)->(0,0) mit nichttrivialer Umwicklung erreicht."""
    start=(0,0,0,0)  # (x,y,wx,wy)
    q=deque([(start,0)])
    visited={start}
    stats={}  # tiefe -> [visited_count, nonlocal_hits]
    while q:
        (x,y,wx,wy),d=q.popleft()
        for dx,dy in STEPS:
            xn_raw,yn_raw=x+dx,y+dy
            wxn,wyn=wx+(xn_raw//L), wy+(yn_raw//L)
            xn,yn=xn_raw%L, yn_raw%L
            depth=d+1
            if record_levels:
                if depth not in stats: stats[depth]=[0,0]
                stats[depth][0]+=1
            # Abschluss: geschlossen & nichtlokal → durch BFS minimal
            if (xn,yn)==(0,0) and (wxn,wyn)!=(0,0):
                if record_levels: stats[depth][1]+=1
                return depth, stats
            state=(xn,yn,wxn,wyn)
            if state in visited: continue
            visited.add(state)
            q.append((state, depth))
    raise RuntimeError("Kein nichtlokaler Zyklus gefunden.")

def witness_path(L):
    """Expliziter Zeuge: L mal +e1 → (0,0) mit (w_x,w_y)=(1,0)."""
    return [(1,0)]*L

def apply_path(L, path):
    """Simuliere Pfad auf T_{L,L} und zähle Umwicklungen (w_x,w_y)."""
    x=y=wx=wy=0
    for dx,dy in path:
        xn_raw,yn_raw=x+dx,y+dy
        wx+=xn_raw//L; wy+=yn_raw//L
        x,y=xn_raw%L, yn_raw%L
    return x,y,wx,wy

# Berechnungen für L=3..15
rows=[]; details={}
for L in range(3,16):
    # Witness
    path=witness_path(L)
    end=apply_path(L,path)
    witness_ok=(end==(0,0,1,0))
    # BFS
    minlen, stats = min_nonlocal_length_bfs(L, record_levels=True)
    # Korrekte Negativ-Aussage: keine nichtlokalen Treffer für Tiefen < L
    no_hits_below_L = all((hits==0) for depth,(vis,hits) in stats.items() if depth < L)
    # Sanity für L=11: Beweislage erzwingen
    if L==11:
        assert minlen==11, "BFS-Minimum muss 11 sein."
        assert witness_ok, "Witness (11×e1) muss (0,0) und (1,0) liefern."
        assert no_hits_below_L, "Erwartet: keine nichtlokalen Treffer für Tiefen < 11."
    rows.append((L,minlen,witness_ok,no_hits_below_L))
    details[L]=stats

df=pd.DataFrame(rows,columns=["L","min_len_bfs","witness_ok","no_hits_below_L"])

# CSV & Hash (Nachvollziehbarkeit)
csv_path = "anc/certificate_alpha11.csv"
csv_text = df.to_csv(index=False)

# Markdown-Tabelle
md_table = "|L|min_len_bfs|witness_ok|no_hits_below_L|\n|:-:|:-:|:-:|:-:|\n"
for L,ml,wok,nh in rows:
    md_table += "|{}|{}|{}|{}|\n".format(L,ml,wok,nh)

# Witness & BFS-Level-Stats für L=11
witness_str = " ".join(str(s) for s in witness_path(11))
stats11 = details[11]
stats_lines = "Tiefe|visited|nonlocal_hits\n:--:|:--:|:--:\n"
for d in sorted(stats11):
    vis,hits=stats11[d]
    stats_lines += "{}|{}|{}\n".format(d,vis,hits)

# CSV (inline)
csv_block = "L,min_len_bfs,witness_ok,no_hits_below_L\n" + "\n".join("{},{},{},{}".format(*row) for row in rows)

with open(csv_path, "w") as f:
    f.write(csv_text)
sha = write_sha256(csv_path)

# Sichtbares Zertifikat (reiner String, keine f-Strings)
snippet  = "### Zertifikat \"11\" – Sichtbarer Beweis\n\n"
snippet += "**A. Schranke:** Jeder nichtlokale Zyklus braucht ≥ L Schritte (Kokette-Argument).\n\n"
snippet += "**B. Witness für L=11:** Pfad = 11×(+e1). Ergebnis: (0,0) mit Umwicklung (1,0).\n"
snippet += "```text\n"+witness_str+"\n```\n\n"
snippet += "**C. BFS-Zertifikat:** Für L=11 keine nichtlokalen Treffer bei Tiefe <11, erster Treffer bei 11.\n"
snippet += "```text\n"+stats_lines+"```\n\n"
snippet += "**Gesamtergebnisse (3≤L≤15):**\n"+md_table+"\n"
snippet += "**CSV (inline):**\n```csv\n"+csv_text+"```\n\n"
snippet += "**Hash (SHA-256 der CSV):**\n`"+sha+"`\n\n"
snippet += "**Beweis-Legende (kompakt):**\n"
snippet += "- *Minimalität durch BFS:* Erste nichtlokale Rückkehr ist per BFS definitionsgemäss die kürzeste.\n"
snippet += "- *Keine kürzeren Gegenbeispiele:* Level-Stats zeigen für L=11 bis Tiefe 10 exakt 0 nichtlokale Treffer.\n"
snippet += "- *Existenzzeugnis:* Der explizite Witness (11×+e1) realisiert die nichtlokale Umwicklung bei Länge 11.\n"

print(snippet)

print("Zertifikat geschrieben:", csv_path)
print("SHA-256:", sha)


### Zertifikat "11" – Sichtbarer Beweis

**A. Schranke:** Jeder nichtlokale Zyklus braucht ≥ L Schritte (Kokette-Argument).

**B. Witness für L=11:** Pfad = 11×(+e1). Ergebnis: (0,0) mit Umwicklung (1,0).
```text
(1, 0) (1, 0) (1, 0) (1, 0) (1, 0) (1, 0) (1, 0) (1, 0) (1, 0) (1, 0) (1, 0)
```

**C. BFS-Zertifikat:** Für L=11 keine nichtlokalen Treffer bei Tiefe <11, erster Treffer bei 11.
```text
Tiefe|visited|nonlocal_hits
:--:|:--:|:--:
1|6|0
2|36|0
3|72|0
4|108|0
5|144|0
6|180|0
7|216|0
8|252|0
9|288|0
10|324|0
11|1|1
```

**Gesamtergebnisse (3≤L≤15):**
|L|min_len_bfs|witness_ok|no_hits_below_L|
|:-:|:-:|:-:|:-:|
|3|3|True|True|
|4|4|True|True|
|5|5|True|True|
|6|6|True|True|
|7|7|True|True|
|8|8|True|True|
|9|9|True|True|
|10|10|True|True|
|11|11|True|True|
|12|12|True|True|
|13|13|True|True|
|14|14|True|True|
|15|15|True|True|

**CSV (inline):**
```csv
L,min_len_bfs,witness_ok,no_hits_below_L
3,3,True,True
4,4,True,True
5,5,True,True
6,6,True,True
7,7,True,True
8,8,True,True
9,9,T

In [9]:
# ========================================
# LaTeX-Export – Dichte/Alpha/Geometrie Labels (kompakt)
# schreibt: labels/dichte_labels.tex
# ========================================

try: ALPHA_THEO
except NameError as e:
    raise RuntimeError("Bitte obige Zellen zuerst ausführen.") from e

import os
from typing import Optional

def _latex_num(x: "mp.mpf", digits: int = 12) -> str:
    return mp.nstr(x, digits)

def _cmd(name: str, value: str) -> str:
    return rf"\providecommand{{\{name}}}{{{value}}}"

def _cmd_if(name: str, value_opt: Optional["mp.mpf"], digits: int = 12) -> list[str]:
    return [_cmd(name, _latex_num(value_opt, digits))] if value_opt is not None else []

# ---- Werte zusammenstellen (nur α, nicht α^{-1}) ----
lines = [
    "% ========================================",
    "% Auto-generierte Labels (Dichte/Geometrie/Alpha)",
    "% Quelle: Notebook-Export",
    "% ========================================\n",
    "% Alpha-Referenzen / Theorie",
    _cmd("AlphaRefPDG",  _latex_num(ALPHA_PDG_2024.v, 12)),
    *_cmd_if("AlphaRefPDGSig",  ALPHA_PDG_2024.sigma, 12),
    _cmd("AlphaRefCOD",  _latex_num(ALPHA_CODATA_2022.v, 12)),
    *_cmd_if("AlphaRefCODSig",  ALPHA_CODATA_2022.sigma, 12),
    _cmd("AlphaTheo",    _latex_num(ALPHA_THEO.v, 12)),
    "",
    "% Geometrie",
    _cmd("EtaFCC", _latex_num(ETA_FCC.v, 12)),
    _cmd("Eta1",   _latex_num(ETA1.v, 12)),
    _cmd("Eta2",   _latex_num(ETA2.v, 12)),
    _cmd("EtaEffEM", _latex_num(ETA_EFF_EM.v, 12)),
    "",
    "% Zählraten (n)",
    _cmd("n1Base", _latex_num(n1_BASE.v, 6)),
    _cmd("n2Base", _latex_num(n2_BASE.v, 6)),
    _cmd("n3Base", _latex_num(n3_BASE.v, 6)) if "n3_BASE" in globals() else "% n3_BASE fehlt",
    _cmd("n0FromParts", _latex_num(n0_FROM_PARTS.v, 6)) if "n0_FROM_PARTS" in globals() else "% n0_FROM_PARTS fehlt",
    "",
    "% Planck/Modell-Dichten",
    _cmd("RhoP", _latex_num(RHO_PLANCK.v, 6)),
    _cmd("RhoModelN8", _latex_num(RHO_MODEL.v, 12)) if "RHO_MODEL" in globals() else "% RHO_MODEL fehlt",
    "",
    "% Kosmologische Referenz-Dichten",
    _cmd("RhoRefPlanck2018", _latex_num(RHO_PLANCK2018.v, 10)),
    *_cmd_if("RhoRefPlanck2018Sig", RHO_PLANCK2018.sigma, 10),
    _cmd("RhoRefWMAP9", _latex_num(RHO_WMAP9.v, 10)),
    *_cmd_if("RhoRefWMAP9Sig", RHO_WMAP9.sigma, 10),
    _cmd("RhoRefSH0ES2021", _latex_num(RHO_SH0ES.v, 10)),
    *_cmd_if("RhoRefSH0ES2021Sig", RHO_SH0ES.sigma, 10),
    "",
]

latex_text = "\n".join(lines) + "\n"

# Schreiben (ohne externe Helfer)
os.makedirs("labels", exist_ok=True)
out_path = "labels/dichte_labels.tex"
with open(out_path, "w", encoding="utf-8") as f:
    f.write(latex_text)

print(f"[OK] LaTeX-Labels geschrieben: {out_path}")
print(latex_text)

[OK] LaTeX-Labels geschrieben: labels/dichte_labels.tex
% ========================================
% Auto-generierte Labels (Dichte/Geometrie/Alpha)
% Quelle: Notebook-Export
% ========================================

% Alpha-Referenzen / Theorie
\providecommand{\AlphaRefPDG}{0.00729735256428}
\providecommand{\AlphaRefPDGSig}{4.26e-13}
\providecommand{\AlphaRefCOD}{0.0072973525643}
\providecommand{\AlphaRefCODSig}{1.1e-12}
\providecommand{\AlphaTheo}{0.00729735256421}

% Geometrie
\providecommand{\EtaFCC}{0.740480489693}
\providecommand{\Eta1}{0.49696969697}
\providecommand{\Eta2}{0.680174761588}
\providecommand{\EtaEffEM}{0.60911198855}

% Zählraten (n)
\providecommand{\n1Base}{1.0}
\providecommand{\n2Base}{1.0}
\providecommand{\n3Base}{135.0}
\providecommand{\n0FromParts}{137.0}

% Planck/Modell-Dichten
\providecommand{\RhoP}{5.15485e+96}
\providecommand{\RhoModelN8}{8.53471982711e-27}

% Kosmologische Referenz-Dichten
\providecommand{\RhoRefPlanck2018}{8.532855164e-27}
\providecomm